# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html

Important note! Some of these models are not yet released in version "0.3.3" which is currently the most recent version on pypi from where you install packages via `pip install`. Hence, you should follow **Install from Source Code** (which I did) **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [1]:
import gluonts
print(gluonts.__version__)

0.4.1


In [2]:
# if version does not include DeepState, uncomment the following:
# !pip install git+https://github.com/awslabs/gluon-ts.git


# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [3]:
def deep_state(seed=42, data="m4_quarterly", epochs=100, batches=50):
    dataset = get_dataset(data)

    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
#         ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         past_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed

    pprint(output)


In [4]:
%%time
res = deep_state(data="m4_weekly", seed=42, epochs=100, batches=50)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.
INFO:root:Using CPU
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [04:25<00:00,  5.30s/it, avg_epoch_loss=0.0381]
INFO:root:Epoch[0] Elapsed time 265.248 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.038141
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [04:22<00:00,  5.25s/it, avg_epoch_loss=-.403]
INFO:root:Epoch[1] Elapsed time 262.630 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-0.402955
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [04:22<00:00,  5.25s/it, avg_epoch_loss=-.806]
INFO:root:Epoch[2] Elapsed time 262.296 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-0.806100
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [04:18<00:00,  5.18s/it, avg_epoch_loss=-.972]
INFO:root:Epoch[3] Elapsed time 258.823 seconds
INFO:root:Epoch[3] Evaluation m

KeyboardInterrupt: 

In [5]:
!jupyter nbconvert --output-dir="./html_outputs" --to html DeepState01-DeepState_experiments_with_gluonts.ipynb

[NbConvertApp] Converting notebook DeepState01-DeepState_experiments_with_gluonts.ipynb to html
[NbConvertApp] Writing 291379 bytes to ./html_outputs\DeepState01-DeepState_experiments_with_gluonts.html


In [7]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deep_state(data="m4_weekly", seed=i, epochs=1, batches=25)
#         pprint(res)
#         results.append(res)
    
# results